In [34]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#dop
import re
from tqdm import tqdm 
from IPython.display import display
from datetime import datetime, timedelta, date, time

#prepprocessing

#ML

#metrics 

# option
pd.set_option("display.max_columns", 300)
plt.rcParams['figure.figsize'] = (15, 10)

PATH_TRAIN = 'data/train.csv'
PATH_TASK = 'data/test.csv'
TARGET_COL = 'target'

In [27]:
time = ['time' + str(i) for i in range(1, 11)]

train = pd.read_csv(PATH_TRAIN, parse_dates=time, index_col='session_id')
task = pd.read_csv(PATH_TASK, parse_dates=time, index_col='session_id')
y_train_full = train[[TARGET_COL]]
df = pd.concat([train.drop(TARGET_COL, axis=1), task], ignore_index=True)

print(train.shape, task.shape)
print('-'*50)
df.info()
df.head()

(253561, 21) (82797, 20)
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336358 entries, 0 to 336357
Data columns (total 20 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   336358 non-null  int64         
 1   time1   336358 non-null  datetime64[ns]
 2   site2   331406 non-null  float64       
 3   time2   331406 non-null  datetime64[ns]
 4   site3   326994 non-null  float64       
 5   time3   326994 non-null  datetime64[ns]
 6   site4   323503 non-null  float64       
 7   time4   323503 non-null  datetime64[ns]
 8   site5   320170 non-null  float64       
 9   time5   320170 non-null  datetime64[ns]
 10  site6   317061 non-null  float64       
 11  time6   317061 non-null  datetime64[ns]
 12  site7   314137 non-null  float64       
 13  time7   314137 non-null  datetime64[ns]
 14  site8   311375 non-null  float64       
 15  time8   311375 non-null  datetime64[ns]
 16

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16
2,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24
3,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42
4,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11


In [30]:
train['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [33]:
pd.date_range(train['time1'], train['time2'])

TypeError: Cannot convert input [session_id
1        2014-02-20 10:02:45
2        2014-02-22 11:19:50
3        2013-12-16 16:40:17
4        2014-03-28 10:52:12
5        2014-02-28 10:53:05
                 ...        
253557   2013-11-25 10:26:54
253558   2013-12-03 16:01:15
253559   2013-12-09 14:05:03
253560   2013-12-19 15:20:22
253561   2014-04-25 09:56:52
Name: time1, Length: 253561, dtype: datetime64[ns]] of type <class 'pandas.core.series.Series'> to Timestamp

---